In [1]:
import numpy as np
import nibabel as nib
import SimpleITK as sitk
from scipy.spatial.distance import directed_hausdorff

In [2]:
# Load the NIfTI files
GTimage1 = sitk.ReadImage('ai4mi_project/data/segthor_train/train/Patient_07/GT_fixed.nii.gz')
GTimage2 = sitk.ReadImage('ai4mi_project/data/segthor_train/train/Patient_17/GT_fixed.nii.gz')
GTimage3 = sitk.ReadImage('ai4mi_project/data/segthor_train/train/Patient_18/GT_fixed.nii.gz')
GTimage4 = sitk.ReadImage('ai4mi_project/data/segthor_train/train/Patient_34/GT_fixed.nii.gz')
GTimage5 = sitk.ReadImage('ai4mi_project/data/segthor_train/train/Patient_39/GT_fixed.nii.gz')

PTimage1 = sitk.ReadImage('ai4mi_project/data/segthor_train/train/ce_rotated_elastic/stitched_volumes/Patient_07.nii.gz')
PTimage2 = sitk.ReadImage('ai4mi_project/data/segthor_train/train/ce_rotated_elastic/stitched_volumes/Patient_17.nii.gz')
PTimage3 = sitk.ReadImage('ai4mi_project/data/segthor_train/train/ce_rotated_elastic/stitched_volumes/Patient_18.nii.gz')
PTimage4 = sitk.ReadImage('ai4mi_project/data/segthor_train/train/ce_rotated_elastic/stitched_volumes/Patient_34.nii.gz')
PTimage5 = sitk.ReadImage('ai4mi_project/data/segthor_train/train/ce_rotated_elastic/stitched_volumes/Patient_39.nii.gz')

In [3]:
# Load NIfTI images
GTimage1_nii = nib.load('ai4mi_project/data/segthor_train/train/Patient_07/GT_fixed.nii.gz')
GTimage2_nii = nib.load('ai4mi_project/data/segthor_train/train/Patient_17/GT_fixed.nii.gz')
GTimage3_nii = nib.load('ai4mi_project/data/segthor_train/train/Patient_18/GT_fixed.nii.gz')
GTimage4_nii = nib.load('ai4mi_project/data/segthor_train/train/Patient_34/GT_fixed.nii.gz')
GTimage5_nii = nib.load('ai4mi_project/data/segthor_train/train/Patient_39/GT_fixed.nii.gz')

PTimage1_nii = nib.load('ai4mi_project/data/segthor_train/train/ce_rotated_elastic/stitched_volumes/Patient_07.nii.gz')
PTimage2_nii = nib.load('ai4mi_project/data/segthor_train/train/ce_rotated_elastic/stitched_volumes/Patient_17.nii.gz')
PTimage3_nii = nib.load('ai4mi_project/data/segthor_train/train/ce_rotated_elastic/stitched_volumes/Patient_18.nii.gz')
PTimage4_nii = nib.load('ai4mi_project/data/segthor_train/train/ce_rotated_elastic/stitched_volumes/Patient_34.nii.gz')
PTimage5_nii = nib.load('ai4mi_project/data/segthor_train/train/ce_rotated_elastic/stitched_volumes/Patient_39.nii.gz')

In [4]:
GTS = [GTimage1, GTimage2, GTimage3, GTimage4, GTimage5]
GTS_nii = [GTimage1_nii, GTimage2_nii, GTimage3_nii, GTimage4_nii, GTimage5_nii]

PTS = [PTimage1, PTimage2, PTimage3, PTimage4, PTimage5]
PTS_nii = [PTimage1_nii, PTimage2_nii, PTimage3_nii, PTimage4_nii, PTimage5_nii]

In [ ]:
def compute_hausdorff(img1, img2):
    """
    img1 has to be the GROUND TRUTH !
    """

    # Convert to numpy arrays
    array1 = sitk.GetArrayFromImage(img1)
    array2 = sitk.GetArrayFromImage(img2)

    # Get all unique labels 
    labels1 = np.unique(array1)
    labels2 = np.unique(array2)
    
    # Dictionary to store Hausdorff distance for each label
    hausdorff_distances = {}

    common_labels = labels1

    # Loop over each label (segmentation)
    for label in common_labels:
        
        if label == 0:  # Skip background
            continue
        
        # Create binary masks for the current label in both images
        mask1 = (array1 == label).astype(int)
        
        if label == 1:
            mask2 = (array2 == labels2[1]).astype(int)
        if label == 2:
            mask2 = (array2 == labels2[2]).astype(int)
        if label == 3:
            mask2 = (array2 == labels2[3]).astype(int)
        if label == 4:
            mask2 = (array2 == labels2[4]).astype(int)
        
        # Get the coordinates of non-zero points in the binary masks
        coords1 = np.column_stack(np.where(mask1))
        coords2 = np.column_stack(np.where(mask2))

        # Compute the directed Hausdorff distances
        hausdorff_distance_1_to_2 = directed_hausdorff(coords1, coords2)[0]
        hausdorff_distance_2_to_1 = directed_hausdorff(coords2, coords1)[0]
    
        # Symmetric Hausdorff distance (maximum of the two directed distances)
        hausdorff_distance = max(hausdorff_distance_1_to_2, hausdorff_distance_2_to_1)
    
        # Store the result
        hausdorff_distances[label] = hausdorff_distance
    
        # print(f"Label {label}: Hausdorff Distance = {hausdorff_distance:.4f}")

    return np.mean(list(hausdorff_distances.values()))

haus = []
for gt, pt in zip(GTS, PTS):
    haus.append(compute_hausdorff(gt, pt))

print("\nMean Hausdorff: ", np.mean(haus))
print("\nSTD Hausdorff: ", np.std(haus))

In [6]:
def compute_volumetric(img1, img2):
    """
    img1 has to be the GROUND TRUTH !
    """

    # Get the data arrays
    mask1 = img1.get_fdata()
    mask2 = img2.get_fdata()
    
    # Get all unique labels from both masks (excluding 0 for the background)
    labels_1 = np.unique(mask1[mask1 > 0])
    labels_2 = np.unique(mask2[mask2 > 0])
    
    # Get the voxel sizes from the headers
    voxel_size1 = np.prod(img1.header.get_zooms())  # in mm^3
    voxel_size2 = np.prod(img2.header.get_zooms())  # in mm^3
    
    # Function to compute volume for a given label
    def compute_volume(mask, label, voxel_size):
        return np.sum(mask == label) * voxel_size
    
    # Iterate over each label (organ) and compute Volumetric Similarity
    vs_results = {}
    
    for label in labels_1:
        # Compute volumes for the current label in both scans
        volume1 = compute_volume(mask1, label, voxel_size1)
    
        if label == 1:
            volume2 = compute_volume(mask2, labels_2[0], voxel_size2)
        if label == 2:
            volume2 = compute_volume(mask2, labels_2[1], voxel_size2)
        if label == 3:
            volume2 = compute_volume(mask2, labels_2[2], voxel_size2)
        if label == 4:
            volume2 = compute_volume(mask2, labels_2[3], voxel_size2)
    
    
        # Compute VS only if both volumes are non-zero
        if volume1 + volume2 > 0:
            vs = 1 - abs(volume1 - volume2) / (volume1 + volume2)
        else:
            vs = 0  # If both volumes are 0, there's no overlap
    
        # Store the result
        vs_results[label] = vs

        # print(f"\nLabel {label}")
        # print(f"Volume img1: {volume1}")
        # print(f"Volume img2: {volume2}\n")
    
    # Print Volumetric Similarity results for each organ
    # for label, vs in vs_results.items():
    #     print(f"Label {label}: Volumetric Similarity = {vs:.4f}")
    # print("i")
    return np.mean(list(vs_results.values()))

# vol = []
# for gt, pt in zip(GTS_nii, PTS_nii):
#     vol.append(compute_volumetric(gt, pt))

# print("\nMean volume: ", np.mean(vol))
# print("\nSTD volume: ", np.std(vol))

# I have to compute the volumetric separetly because my kernel died

In [8]:
vol1 = compute_volumetric(GTS_nii[0], PTS_nii[0])
print(vol1)


0.9802115767442802


In [9]:
vol2 = compute_volumetric(GTS_nii[1], PTS_nii[1])
print(vol2)


0.893861309773059


In [10]:
vol3 = compute_volumetric(GTS_nii[2], PTS_nii[2])
print(vol3)


0.9344621027255262


In [7]:
vol4 = compute_volumetric(GTS_nii[3], PTS_nii[3])
print(vol4)


0.8351096432126663


In [8]:
vol5 = compute_volumetric(GTS_nii[4], PTS_nii[4])
print(vol5)


0.9547427561056475


In [9]:
vol = [0.9802115767442802, 0.893861309773059, 0.9344621027255262, 0.8351096432126663, 0.9547427561056475]
#vol = [vol1, vol2, vol3, vol4, vol5]

print("\nMean volume: ", np.mean(vol))
print("\nSTD volume: ", np.std(vol))


Mean volume:  0.9196774777122358

STD volume:  0.050854162445610654


In [ ]:
def compute_confusion_metrics(img1, img2):
    """
    img1 has to be the GROUND TRUTH !
    """

    seg1 = img1.get_fdata()
    seg2 = img2.get_fdata()
    
    labels_1 = np.unique(seg1[seg1 > 0])
    labels_2 = np.unique(seg2[seg2 > 0])
    my_dict = dict(zip(labels_1, labels_2))
    
    # Ensure both segmentations have the same shape
    assert seg1.shape == seg2.shape, "The two segmentations must have the same shape."
    
    # Convert the 3D arrays into 1D arrays for easy comparison
    seg1_flat = seg1.flatten()
    seg2_flat = seg2.flatten()
    
    for label in labels_1:
    
        # Compute True Positives, True Negatives, False Positives, and False Negatives
        TP = np.sum((seg1_flat == label) & (seg2_flat == my_dict[label]))  # Both true
        TN = np.sum((seg1_flat != label) & (seg2_flat != my_dict[label]))  # Both false
        FP = np.sum((seg1_flat != label) & (seg2_flat == my_dict[label]))  # False positive: seg2 says 1 but seg1 says 0
        FN = np.sum((seg1_flat == label) & (seg2_flat != my_dict[label]))  # False negative: seg2 says 0 but seg1 says 1
        
        # Total number of voxels
        total_voxels = seg1_flat.size
        
        # Calculate percentages
        TP_percent = (TP / total_voxels) * 100
        TN_percent = (TN / total_voxels) * 100
        FP_percent = (FP / total_voxels) * 100
        FN_percent = (FN / total_voxels) * 100
        
        # Output the results in percentages
        print(f'\nResults for Label: {label}, Total voxels for this label: {np.sum(seg1_flat == label)}')
        print(f'True Positives (TP): {TP_percent:.4f}%, TP voxels: {TP}')
        print(f'True Negatives (TN): {TN_percent:.4f}%, TN voxels: {TN}')
        print(f'False Positives (FP): {FP_percent:.4f}%, FP voxels: {FP}')
        print(f'False Negatives (FN): {FN_percent:.4f}%, FN voxels: {FN}')

# compute_confusion_metrics(nii_1, nii_2)

In [6]:
def compute_average(img1, img2):
    """
    img1 has to be the GROUND TRUTH !
    """

    # Convert to numpy arrays for easier manipulation
    seg1_np = sitk.GetArrayFromImage(img1)
    seg2_np = sitk.GetArrayFromImage(img2)
    
    # Find all unique labels (excluding background)
    labels1 = np.unique(seg1_np)
    labels2 = np.unique(seg2_np) 
    
    average_surface_distances = []
    labels = labels1[1:]
    
    for label in labels:
    
        # Extract binary masks for the current label in both segmentations
        seg1_mask = (seg1_np == label).astype(np.uint8)
        seg2_mask = (seg2_np == label).astype(np.uint8)
    
        if label == 1:
            seg2_mask = (seg2_np == labels2[1]).astype(np.uint8)
        if label == 2:
            seg2_mask = (seg2_np == labels2[2]).astype(np.uint8)
        if label == 3:
            seg2_mask = (seg2_np == labels2[3]).astype(np.uint8)
        if label == 4:
            seg2_mask = (seg2_np == labels2[4]).astype(np.uint8)
    
        # Convert the masks back to SimpleITK images
        seg1_mask_img = sitk.GetImageFromArray(seg1_mask)
        seg2_mask_img = sitk.GetImageFromArray(seg2_mask)
    
        # Compute Average Surface Distance
        seg1_surface = sitk.LabelContour(seg1_mask_img)
        seg2_surface = sitk.LabelContour(seg2_mask_img)
    
        surface_distance_filter = sitk.HausdorffDistanceImageFilter()
        surface_distance_filter.Execute(seg1_surface, seg2_surface)
        avg_distance = surface_distance_filter.GetAverageHausdorffDistance()
        average_surface_distances.append(avg_distance)
    
    # Output results for each label
    # for i, label in enumerate(labels):
    #     print(f"Label {label}:")
    #     print(f"  Average Surface Distance: {average_surface_distances[i]}")
    return np.mean(average_surface_distances)

avg = []
for gt, pt in zip(GTS, PTS):
    avg.append(compute_average(gt, pt))

print("\nMean avg: ", np.mean(avg))
print("\nSTD avg: ", np.std(avg))


Mean avg:  2.5389224428466357

STD avg:  0.5194738784552952
